In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer


In [7]:
model = SentenceTransformer("all-mpnet-base-v2")

# Our sentences to encode
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog."
]

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\myFiles\personal\Yama\.env\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Faraz Ahmadi\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformers.jso

Sentence: This framework generates embeddings for each input sentence
Embedding: [ 6.41695224e-03  7.04139424e-03 -2.81442031e-02  5.12470901e-02
 -8.93962104e-03  2.12668926e-02  2.30779275e-02 -1.44859804e-02
 -5.55315148e-03 -2.49296967e-02  4.53493483e-02  2.48959139e-02
 -3.07578687e-02  5.66224568e-02  6.32021725e-02 -5.62527776e-02
  5.16509786e-02  5.78274857e-03 -2.62116976e-02  1.31871295e-03
  1.99272558e-02 -1.30589562e-03 -2.28704163e-03  4.72541489e-02
 -3.72494720e-02 -2.85245050e-02 -4.10240851e-02 -1.57976318e-02
  3.17329657e-03 -8.74146353e-04 -2.96460167e-02  3.21501680e-02
  3.51344310e-02  1.09737925e-02  9.16707108e-07 -1.18584989e-03
 -2.53641233e-02 -7.92883150e-03 -5.09482855e-03  7.40656676e-03
  2.80068424e-02  1.06995171e-02  1.07513545e-02  2.76827812e-02
 -5.19132279e-02 -4.98179086e-02  5.34074381e-02  5.79066500e-02
  7.86073804e-02  7.73013905e-02 -1.01112006e-02 -6.35446087e-02
 -1.71579998e-02 -6.77371258e-03 -2.45819031e-03  2.61346083e-02
 -5.38507

In [8]:
embeddings.shape

(3, 768)

In [34]:
import requests
from secret import HUGGING_FACE_API_KEY
import re
API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
headers = {"Authorization": f"Bearer {HUGGING_FACE_API_KEY}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
		"source_sentence": "That is a happy person",
		"sentences": [
			"That is a happy dog",
			"That is a very happy person",
			"Today is a sunny day",
			"He is a happy person",
		]
	},
})

In [35]:
output

[0.6945773959159851,
 0.9429152011871338,
 0.2568760812282562,
 0.7561243176460266]

In [41]:
# FILEPATH: /c:/myFiles/personal/Yama/claim_sim.ipynb
claims = pd.read_excel("data/Claims_Variations.xlsx")
claims.head(10)

,claim_id,main_claim,variations
0,0,"For adults with T2D and established CVD, to re...","1. ""Reduce the risk of major cardiovascular ev..."
1,0,"For adults with T2D and established CVD, to re...","2. ""Minimize the likelihood of major adverse c..."
2,0,"For adults with T2D and established CVD, to re...","3. ""Help lower the chances of major cardiovasc..."
3,0,"For adults with T2D and established CVD, to re...","4. ""Decrease the risk of major adverse cardiov..."
4,0,"For adults with T2D and established CVD, to re...","5. ""Protect against major cardiovascular event..."
5,1,Ozempic® (semaglutide) is the #1 prescribed br...,1. Ozempic® (semaglutide) is the top prescribe...
6,1,Ozempic® (semaglutide) is the #1 prescribed br...,"2. As the leading branded medication for T2D, ..."
7,1,Ozempic® (semaglutide) is the #1 prescribed br...,3. With its proven effectiveness and widesprea...
8,1,Ozempic® (semaglutide) is the #1 prescribed br...,4. Ozempic® (semaglutide) is the most commonly...
9,1,Ozempic® (semaglutide) is the #1 prescribed br...,5. As the preferred branded medication for T2D...


In [57]:
def clean_string(s):
    # remove leading numbers and periods
    s = re.sub(r'^\d+\.\s*', '', s)
    # extract text within quotes
    match = re.search(r'"([^"]*)"', s)
    if match:
        var = match.group(1)
    else:
        var = s
    return var

cosine_simil = []
for claim_id in claims['claim_id'].unique():
    claim_subset = claims[claims['claim_id'] == claim_id].copy()
    claim_subset['variations'] = claim_subset['variations'].apply(clean_string)
    # print(claim_subset['variations'].to_list())
    send_dict = {"source_sentence": claim_subset.main_claim.iloc[0], "sentences": claim_subset['variations'].to_list()}
    output = query({"inputs": send_dict})
    cosine_simil.append(output)



In [59]:
cosine_simil

[[0.44236648082733154,
  0.3487890064716339,
  0.41104450821876526,
  0.4013262689113617,
  0.4219215214252472],
 [0.8772132396697998,
  0.855892539024353,
  0.8781327605247498,
  0.8383111953735352,
  0.8500670790672302],
 [0.436561644077301,
  0.350335955619812,
  0.45905932784080505,
  0.3630131781101227,
  0.158079132437706],
 [0.6836378574371338,
  0.7895714640617371,
  0.49884167313575745,
  0.597878634929657,
  0.6246694922447205],
 [0.5361419916152954,
  0.6354564428329468,
  0.5274009108543396,
  0.6606813669204712,
  0.5295681357383728],
 [0.12119068950414658,
  0.10550297051668167,
  0.07114662975072861,
  0.054350219666957855,
  0.07844088226556778],
 [0.7022290229797363,
  0.6167792081832886,
  0.6690917611122131,
  0.9099352359771729,
  0.6449251174926758],
 [0.881389856338501,
  0.897774338722229,
  0.8975157737731934,
  0.9372667074203491,
  0.7422432899475098],
 [0.8260341882705688,
  0.7356550693511963,
  0.8402770757675171,
  0.8222359418869019,
  0.7796039581298828]

In [58]:
claims['cosine_simil'] = cosine_simil
claims.head(10)

ValueError: Length of values (28) does not match length of index (140)

In [45]:

for index, row in claims.iterrows():
    print(row.main_claim)
    print(type(row.variations))

    match = re.search(r'"([^"]*)"', row.variations)
    if match:
        var = match.group(1)
    else:
        var = row.variations
    send_dict = {"source_sentence": row.main_claim, "sentences": [var]}
    output = query({"inputs": send_dict})
    row["similarity"] = output[0]
claims

For adults with T2D and established CVD, to reduce risk of MACE
<class 'str'>
For adults with T2D and established CVD, to reduce risk of MACE
<class 'str'>
For adults with T2D and established CVD, to reduce risk of MACE
<class 'str'>
For adults with T2D and established CVD, to reduce risk of MACE
<class 'str'>
For adults with T2D and established CVD, to reduce risk of MACE
<class 'str'>
Ozempic® (semaglutide) is the #1 prescribed branded diabetes medication for new prescriptionsa
<class 'str'>
Ozempic® (semaglutide) is the #1 prescribed branded diabetes medication for new prescriptionsa
<class 'str'>
Ozempic® (semaglutide) is the #1 prescribed branded diabetes medication for new prescriptionsa
<class 'str'>
Ozempic® (semaglutide) is the #1 prescribed branded diabetes medication for new prescriptionsa
<class 'str'>
Ozempic® (semaglutide) is the #1 prescribed branded diabetes medication for new prescriptionsa
<class 'str'>
PROVEN CV RISK REDUCTION
<class 'str'>
PROVEN CV RISK REDUCTION
<

,claim_id,main_claim,variations
0,0,"For adults with T2D and established CVD, to re...","1. ""Reduce the risk of major cardiovascular ev..."
1,0,"For adults with T2D and established CVD, to re...","2. ""Minimize the likelihood of major adverse c..."
2,0,"For adults with T2D and established CVD, to re...","3. ""Help lower the chances of major cardiovasc..."
3,0,"For adults with T2D and established CVD, to re...","4. ""Decrease the risk of major adverse cardiov..."
4,0,"For adults with T2D and established CVD, to re...","5. ""Protect against major cardiovascular event..."
...,...,...,...
135,27,AEs occurring in ≥5% of participants treated w...,"1. ""Ozempic® effectively managed T2D with a lo..."
136,27,AEs occurring in ≥5% of participants treated w...,"2. ""SUSTAIN 5 study shows Ozempic® has a favor..."
137,27,AEs occurring in ≥5% of participants treated w...,"3. ""Significant reduction in T2D symptoms seen..."
138,27,AEs occurring in ≥5% of participants treated w...,"4. ""Ozempic® demonstrated a high safety profil..."


In [46]:
claims

,claim_id,main_claim,variations
0,0,"For adults with T2D and established CVD, to re...","1. ""Reduce the risk of major cardiovascular ev..."
1,0,"For adults with T2D and established CVD, to re...","2. ""Minimize the likelihood of major adverse c..."
2,0,"For adults with T2D and established CVD, to re...","3. ""Help lower the chances of major cardiovasc..."
3,0,"For adults with T2D and established CVD, to re...","4. ""Decrease the risk of major adverse cardiov..."
4,0,"For adults with T2D and established CVD, to re...","5. ""Protect against major cardiovascular event..."
...,...,...,...
135,27,AEs occurring in ≥5% of participants treated w...,"1. ""Ozempic® effectively managed T2D with a lo..."
136,27,AEs occurring in ≥5% of participants treated w...,"2. ""SUSTAIN 5 study shows Ozempic® has a favor..."
137,27,AEs occurring in ≥5% of participants treated w...,"3. ""Significant reduction in T2D symptoms seen..."
138,27,AEs occurring in ≥5% of participants treated w...,"4. ""Ozempic® demonstrated a high safety profil..."
